In [5]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import psycopg2
import psycopg2.extras
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import urllib
import json
import operator
import ast
import random
from scipy.stats import norm
from rcspp import RSCPP_LS
from gen_col import GEN_COL

In [6]:
##### note: LAND_COST needs to be updated in rcspp.py

In [7]:
# pVector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0]
# rscpp_l5_fp = RSCPP_LS(5, 0, pVector)
# rscpp_l5_fp.find_demand()
# rscpp_l5_fp.model_rscpp()
# rscpp_l5_fp.solve_rscpp()
# rscpp_l5_fp.exp_excess_rev

In [8]:
#pVector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0]

In [9]:
runtime = 0.0

In [10]:
# number of services
numS = 3
S = range(numS)

In [11]:
def find_exp_excess_rev(location, assignment_vector):
    global runtime
    # total expected excess revenue for this location given this assignment vector
    eer = 0.0
    for s in S:
        rscpp = RSCPP_LS(location, s, assignment_vector)
        rscpp.find_demand()
        rscpp.model_rscpp()
        rscpp.solve_rscpp()
        eer += rscpp.exp_excess_rev
        runtime += rscpp.results['Solver'][0]['User time']
    return eer

In [12]:
def find_capacity_configuration(location, assignment_vector, count_time):
    global runtime
    # total expected excess revenue for this location given this assignment vector
    eer = 0.0
    people_capacity = {}
    building_capacity = {}
    for s in S:
        people_capacity[s] = []
        building_capacity[s] = []
    for s in S:
        rscpp = RSCPP_LS(location, s, assignment_vector)
        rscpp.find_demand()
        rscpp.model_rscpp()
        rscpp.solve_rscpp()
        eer += rscpp.exp_excess_rev
        if count_time:
            runtime += rscpp.results['Solver'][0]['User time']
        for t in rscpp.model.T:
            people_capacity[s].append(value(rscpp.model.p_tilde[t]))
            building_capacity[s].append(value(rscpp.model.b_tilde[t]))
    #print rscpp.results
    return eer, people_capacity, building_capacity

In [13]:
# pVector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0]
# eer, people_capacity, building_capacity = find_capacity_configuration(5, pVector)

In [14]:
# people_capacity

In [15]:
# l = 5
# pc = pd.DataFrame(data = people_capacity)
# pc.rename(columns = {0:'FP', 1:'IM', 2:'P'}, inplace = True)
# pc['location'] = l
# pc

In [16]:
bfs_tree = pd.read_csv('bfs_tree.csv',index_col=False)
bfs_tree.set_index('p', inplace = True)
bfs_tree.columns= bfs_tree.columns.astype(int)
bfs_tree

,1,2,3,4,5,6,7,8
p,,,,,,,,
1,2,3,4,2,3,2,2,3
2,2,3,4,2,3,3,3,3
3,3,4,5,3,4,3,3,4
4,3,5,4,4,5,0,1,4
5,3,4,3,3,4,1,1,3
6,2,4,4,3,4,2,2,4
7,1,4,5,3,4,3,3,4
8,2,5,4,4,5,2,2,4
9,3,5,4,4,5,2,1,4


In [17]:
numL = len(bfs_tree.columns)
numL

8

In [18]:
distance = pd.read_csv('distance.csv',index_col=False)
distance.set_index('p', inplace = True)
distance.columns= bfs_tree.columns.astype(int)
distance

,1,2,3,4,5,6,7,8
p,,,,,,,,
1,3.07,5.33,5.64,6.10,4.67,4.01,3.52,5.85
2,3.03,4.98,5.30,5.75,4.36,5.75,5.05,5.61
3,6.93,9.23,9.55,10.00,8.61,8.05,7.60,9.86
4,1.78,3.03,2.54,3.73,2.51,0.38,0.80,3.48
5,4.08,6.38,5.78,7.14,5.82,2.75,2.33,6.97
6,1.92,4.22,4.56,5.02,3.62,2.61,2.13,4.81
7,1.08,3.38,3.73,4.18,2.79,1.64,1.53,4.18
8,1.74,4.08,4.39,5.89,3.41,1.74,1.29,4.56
9,2.20,4.53,4.84,5.30,3.87,1.11,0.91,5.16


In [19]:
columns = {}
for l in range(1, numL+1):
    columns[l] = {}
#columns = [{}]*numL
columns

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}}

In [20]:
# pVector is the column to enter, obj_val is the expected excess revenue associated with the entering column
def add_a_column(master_prob, pVector, obj_val,l):
    # add a new variable in the location block, obtain the index of the newly added variable
    master_prob.block_l[l].v.add()
    vind = len(master_prob.block_l[l].v)
    master_prob.block_l[l].v[vind].setlb(0)
    master_prob.block_l[l].v[vind].setub(1)
    master_prob.block_l[l].constr._body += master_prob.block_l[l].v[vind]
    getattr(master_prob, 'obj').set_value(getattr(master_prob, 'obj').expr + obj_val * master_prob.block_l[l].v[vind])
    for p in master_prob.P:
        master_prob.cover_pop_center[p]._body += pVector[p-1] * master_prob.block_l[l].v[vind]
    #columns[l-1].update({vind: list(pVector)})
    #print ('l=', l, ', before_add_pvector', columns[l-1])
    columns[l][vind] = list(pVector)
    #print ('l=', l,', after_add_pvector', columns[l-1])


In [21]:
############## step 0. Construcct master prob that holds all the columns ##########
master_prob = ConcreteModel()

######### paramters #########
# number of population centers
master_prob.numP = Param(within = NonNegativeIntegers, initialize = len(bfs_tree))

# number of facilities
master_prob.numL = Param(within = NonNegativeIntegers, initialize = len(bfs_tree.columns))

# paramter as value of 1 to hold place in the initialized constraint -1 <= 0
master_prob.one = Param(within = NonNegativeReals, initialize = 1, mutable = True)

########## sets ############
# set of population centers
master_prob.P = RangeSet(master_prob.numP, ordered=True)
# set of facility locations
master_prob.L = RangeSet(master_prob.numL, ordered=True)

# blocks of variables
def location_block_rule(b, l):
    b.v = VarList()
    b.v0 = Var(within = NonNegativeReals, bounds=(0,1))
    b.constr = Constraint(expr = b.v0 - master_prob.one == 0)
master_prob.block_l = Block(master_prob.L, rule = location_block_rule)

########## constraints ###########
def cover_pop_center_rule(master_prob, p):
    return  -master_prob.one <= 0
master_prob.cover_pop_center = Constraint(master_prob.P, rule = cover_pop_center_rule)


########### objective #############
master_prob.obj = Objective(expr = 0.0, sense = maximize)

########### dual variables ##############
master_prob.dual = Suffix(direction=Suffix.IMPORT)


In [22]:
########## setp 1: initialization #############
# for each location, add a column that covers population centers up to depth d
for depth in [0,1]:
#for depth in [0]:
    for l in master_prob.L:
        # find the assignment vector
        pv = [int(i<=depth) for i in bfs_tree[l].copy()]
        # compute expected excess revenue of this vector
        obj_val = find_exp_excess_rev(l, pv)
        print ("l= {0}, depth= {1}, obj = {2}".format(l, depth, obj_val))
        add_a_column(master_prob, pv, obj_val, l)

maximum_depth = 3
stop = False
iter_num = 0
solver = SolverFactory('cplex')

# test start
#results = solver.solve(master_prob)

# test end

while (stop==False) and (iter_num<=100):
    iter_num += 1
    print ('iteration ={0}, num_vars = {1}'.format(iter_num,  sum(len(master_prob.block_l[l].v) for l in master_prob.L)))
    ######## step 2: solve master problem #######
    results = solver.solve(master_prob)
    runtime += results['Solver'][0]['User time']
    print (value(master_prob.obj))
    # dual variable per population coverage constraint
    w = []
    cobject = getattr(master_prob, 'cover_pop_center')
    for index in cobject:
        w.append(master_prob.dual[cobject[index]])
        
    # dual variable per location's convex combination constraint
    alpha = []
    for l in master_prob.L:
        cobject = getattr(master_prob.block_l[1], 'constr')
        alpha.append(master_prob.dual[cobject])
    
    # slack variable
    slack = []
    for p in master_prob.P:
        slack.append(master_prob.cover_pop_center[p].uslack())
    ########### step 3: iterate CGH_l for each location to find potential entering columns ########
    col_gen_flag = 0
    for l in master_prob.L:
        gen_col_model = GEN_COL(maximum_depth)
        gen_col_model.gen_col_for_a_location(w, alpha[l-1], slack, bfs_tree[l].tolist())
        if len(gen_col_model.col)>0:
            ######## step 4: find expected excess revenue per column ###########
            # expected excess revenue
            exp_excess_rev = find_exp_excess_rev(l, gen_col_model.col)
            # reduced cost
            gen_col_model.check_reduced_cost(w, alpha[l-1], exp_excess_rev)
            ######## step 5: find reduced cost, see if it is a promising col ####
            if gen_col_model.reduced_cost > 0:
                col_gen_flag += 1
                add_a_column(master_prob, gen_col_model.col, exp_excess_rev, l)
    if col_gen_flag == 0:
        stop = True

######## step 6: change master problem to binary problem and solve ######
for l in master_prob.L:
    num_var = len(master_prob.block_l[l].v)
    for k in range(1, num_var+1):
        master_prob.block_l[l].v[k]._domain = Binary

results = solver.solve(master_prob)
runtime += results['Solver'][0]['User time']


# find opened facility locations
open_facility = []
open_facility_pvector = []
num_cols = 0
for l in master_prob.L:
    num_var = len(master_prob.block_l[l].v)
    num_cols += num_var
    for k in range(1, num_var+1):
        if value(master_prob.block_l[l].v[k]) > 0:
            open_facility.append(l)
            open_facility_pvector.append(columns[l][k])
                
# find uncovered population centers
uncovered_pop_centers = []
for p in master_prob.P:
    if master_prob.cover_pop_center[p].uslack() > 0:
        uncovered_pop_centers.append(p)

##### step 7: assign uncovered population centers (if any) to open facilities
final_assigned_vector_exp_excess_rev = []
building_capacity_df = pd.DataFrame()
people_capacity_df = pd.DataFrame()
if len(uncovered_pop_centers) == 0:
    print ('Optimal integer solution covers all population centers. Terminate. Obj = {0}'.format(value(master_prob.obj)))
    for l in open_facility:
        index_of_pvector = open_facility.index(l)
        # call find_capacity_configuration function to find capacity for each service in each open facility
        # do NOT count run time because these columns have been added and solved in RCSPP
        eer, people_capacity, building_capacity = find_capacity_configuration(l, open_facility_pvector[index_of_pvector], False )
        final_assigned_vector_exp_excess_rev.append(eer)
        # construct people capacity table
        pc = pd.DataFrame(data = people_capacity)
        pc.rename(columns = {0:'FP', 1:'IM', 2:'P'}, inplace = True)
        pc['location'] = l
        people_capacity_df = people_capacity_df.append([pc])
        # construct builidng capacity table
        bc = pd.DataFrame(data = building_capacity)
        bc.rename(columns = {0:'FP', 1:'IM', 2:'P'}, inplace = True)
        bc['location'] = l
        building_capacity_df = building_capacity_df.append([bc])
else:
    for p in uncovered_pop_centers:
        # index of the neareast facility to pop center p
        nearest_facility = np.argmin(distance.loc[p][open_facility])
        # assign p to nearest_facility
        open_facility_pvector[open_facility.index(nearest_facility)][p-1] = 1
    for l in open_facility:
        index_of_pvector = open_facility.index(l)
        # call find_capacity_configuration function to 1) solve the exp excess revenue in each final assigned column,
        # and 2) find capacity for each service in each open facility
        # need to Count run time because these columns have been changed and need to re-solve
        eer, people_capacity, building_capacity = find_capacity_configuration(l, open_facility_pvector[index_of_pvector], True )
        final_assigned_vector_exp_excess_rev.append(eer)
        # construct people capacity table
        pc = pd.DataFrame(data = people_capacity)
        pc.rename(columns = {0:'FP', 1:'IM', 2:'P'}, inplace = True)
        pc['location'] = l
        people_capacity_df = people_capacity_df.append([pc])
        # construct builidng capacity table
        bc = pd.DataFrame(data = building_capacity)
        bc.rename(columns = {0:'FP', 1:'IM', 2:'P'}, inplace = True)
        bc['location'] = l
        building_capacity_df = building_capacity_df.append([bc])

        final_obj_value = sum(rev for rev in final_assigned_vector_exp_excess_rev)
        print ('Assigned uncovered population centers to open facilities. Terminate. Obj = {0}'.format(final_obj_value))

l= 1, depth= 0, obj = -653400.0
l= 2, depth= 0, obj = -269496.0
l= 3, depth= 0, obj = -174240.0
l= 4, depth= 0, obj = 16323238.0
l= 5, depth= 0, obj = -174240.0
l= 6, depth= 0, obj = -78853.0
l= 7, depth= 0, obj = -27443.0
l= 8, depth= 0, obj = 11836659.0
l= 1, depth= 1, obj = 42223220.0
l= 2, depth= 1, obj = 73592037.0
l= 3, depth= 1, obj = 73178320.0
l= 4, depth= 1, obj = 121114635.0
l= 5, depth= 1, obj = 14642149.0
l= 6, depth= 1, obj = 44758499.0
l= 7, depth= 1, obj = 50631800.0
l= 8, depth= 1, obj = 130363258.0
iteration =1, num_vars = 16
237860427.0
iteration =2, num_vars = 24
317826645.5
iteration =3, num_vars = 28
327919977.0
iteration =4, num_vars = 34
348551259.0
iteration =5, num_vars = 38
348551259.0
Assigned uncovered population centers to open facilities. Terminate. Obj = 222147039.0
Assigned uncovered population centers to open facilities. Terminate. Obj = 360735859.0


In [23]:
final_assigned_vector_exp_excess_rev

[222147039.0, 138588820.0]

In [24]:
people_capacity_df

,FP,IM,P,location
0,8.0,8.0,4.0,2
1,8.0,8.0,4.0,2
2,8.0,8.0,5.0,2
3,8.0,8.0,5.0,2
4,10.0,8.0,5.0,2
5,10.0,8.0,5.0,2
6,10.0,8.0,5.0,2
7,10.0,10.0,8.0,2
8,10.0,10.0,8.0,2
9,10.0,10.0,8.0,2


In [25]:
building_capacity_df

,FP,IM,P,location
0,8.0,8.0,4.0,2
1,8.0,8.0,4.0,2
2,8.0,8.0,5.0,2
3,8.0,8.0,5.0,2
4,10.0,8.0,5.0,2
5,10.0,8.0,5.0,2
6,10.0,8.0,5.0,2
7,10.0,10.0,8.0,2
8,10.0,10.0,8.0,2
9,10.0,10.0,8.0,2


In [26]:
num_cols

38

In [27]:
iter_num

5

In [28]:
runtime

37.09999999999998

In [29]:
# find opened facility locations
open_facility = []
open_facility_pvector = []
for l in master_prob.L:
    num_var = len(master_prob.block_l[l].v)
    for k in range(1, num_var+1):
        if value(master_prob.block_l[l].v[k]) > 0:
            open_facility.append(l)
            open_facility_pvector.append(columns[l][k])
                
# find uncovered population centers
uncovered_pop_centers = []
for p in master_prob.P:
    if master_prob.cover_pop_center[p].uslack() > 0:
        uncovered_pop_centers.append(p)

if len(uncovered_pop_centers) == 0:
    print ('Optimal integer solution covers all population centers. Terminate.')
else:
    ##### step 7: assign uncovered population centers to open facilities
    for p in uncovered_pop_centers:
        # index of the neareast facility to pop center p
        nearest_facility = np.argmin(distance.loc[p][open_facility])
        # assign p to nearest_facility
        open_facility_pvector[open_facility.index(nearest_facility)][p-1] = 1
    
    final_assigned_vector_exp_excess_rev = []
    for l in open_facility:
        index_of_pvector = open_facility.index(l)
        final_assigned_vector_exp_excess_rev.append( find_exp_excess_rev(l, open_facility_pvector[index_of_pvector] ))

In [45]:
print('final_total_expected_excess_revenue = {0}'.format(sum(rev for rev in final_assigned_vector_exp_excess_rev)))

final_total_expected_excess_revenue = 360735859.0


- below are codes to check results

In [31]:
open_facility

[2, 6]

In [32]:
open_facility_pvector[open_facility.index(2)]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1,
 1.0,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0]

In [35]:
# np.argmin(distance.loc[34][open_facility])

In [37]:
print ('slack, dual')
for i in range(len(w)):
    print (slack[i], w[i])

slack, dual
(0.0, 44552737.0)
(0.0, 4688868.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(1.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, 50631800.0)
(0.0, -0.0)
(0.0, 35331988.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(1.0, -0.0)
(0.0, -0.0)
(1.0, -0.0)
(1.0, -0.0)
(0.0, 9770379.0)
(0.0, 9185399.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, -0.0)
(0.0, 10365813.0)
(0.0, 19676228.0)
(0.0, 6330658.0)
(0.0, 71032541.0)
(0.0, 11836659.0)
(0.0, 64287286.0)
(0.0, -0.0)
(0.0, 10860903.0)
(0.0, -0.0)
(0.0, -0.0)


In [38]:
master_prob.pprint()

2 RangeSet Declarations
    L : Dim=0, Dimen=1, Size=8, Domain=Integers, Ordered=True, Bounds=(1, 8)
        Virtual
    P : Dim=0, Dimen=1, Size=36, Domain=Integers, Ordered=True, Bounds=(1, 36)
        Virtual

3 Param Declarations
    numL : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None :     8
    numP : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None :    36
    one : Size=1, Index=None, Domain=NonNegativeReals, Default=None, Mutable=True
        Key  : Value
        None :     1

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize :  - 653400.0*block_l[1].v[1] - 269496.0*block_l[2].v[1] - 174240.0*block_l[3].v[1] + 16323238.0*block_l[4].v[1] - 174240.0*block_l[5].v[1] - 78853.0*block_l[6].v[1] - 27443.0*block_l[7].v[1] + 11836659.0*block_l[8].v[1] + 42223220.0*blo